In [1]:
import os
import pandas as pd

sys.path.insert(0, '..\..\Resources\Python-Functions')
import BigQuery

In [2]:
# import transit route outputs
bqclient = BigQuery.getBigQueryClient_TDMScenarios()

In [3]:
dfRouteSummary = bqclient.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_summary AS rs LEFT JOIN tdm-scenarios.tdm_scenarios_output.scenarios AS s ON rs.scenarioID = s.scenarioID;").to_dataframe()
dfRouteSummary

,scenarioID,MODE,NAMEID,NAME,OPERATOR,DIST,PK_HEADWAY,OK_HEADWAY,PK_TIME,OK_TIME,...,DY_D_BRDA,DY_W_BRDA,DY_BRDA,DY_XITB,scenarioID_1,tdmVersion,tdmVersionWithDate,scenarioName,scenarioGroup,scenarioYear
0,8,4,33.0,S1,0,10.72,15,15,47.63,46.09,...,10.78,798.97,809.75,809.75,8,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2032,Needs,2032
1,9,4,29.0,S1,0,10.72,15,15,48.46,47.29,...,4.94,986.69,991.63,991.63,9,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2042,Needs,2042
2,10,4,37.0,S1,0,10.72,15,15,48.72,47.37,...,6.11,1037.45,1043.56,1043.56,10,v832,WF TDM v8.3.2 - 2022-09-21,Needs_2050,Needs,2050
3,14,4,38.0,S1,0,10.72,15,15,47.58,46.10,...,11.41,808.72,820.13,820.13,14,v832,WF TDM v8.3.2 - 2022-09-21,RTP_2032,RTP,2032
4,15,4,36.0,S1,0,10.72,15,15,48.38,47.03,...,5.64,1010.38,1016.02,1016.02,15,v832,WF TDM v8.3.2 - 2022-09-21,RTP_2042,RTP,2042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5245,18,4,17.0,M864_ThanksgivingPnt,0,7.25,30,30,25.56,26.49,...,21.88,77.36,99.24,99.24,18,v9,WF TDM v9.0 - 2022-12-19,BY_2019,BY,2019
5246,26,4,14.0,M864_ThanksgivingPnt,0,7.25,30,30,24.95,26.37,...,26.41,150.14,176.55,176.55,26,v9,WF TDM v9.0 - 2022-12-19,NoBuild_2032,NoBuild,2032
5247,27,4,14.0,M864_ThanksgivingPnt,0,7.25,30,30,25.15,26.47,...,28.72,188.14,216.86,216.86,27,v9,WF TDM v9.0 - 2022-12-19,NoBuild_2042,NoBuild,2042
5248,28,4,14.0,M864_ThanksgivingPnt,0,7.25,30,30,25.47,26.67,...,31.90,209.51,241.41,241.41,28,v9,WF TDM v9.0 - 2022-12-19,NoBuild_2050,NoBuild,2050


In [4]:
# don't need directions
dfRouteSummaryNames = dfRouteSummary.copy()
dfRouteSummaryNames['NAME'] = dfRouteSummaryNames['NAME'].str.replace('-','')
dfRouteSummaryNames = dfRouteSummaryNames.groupby(['NAME'],as_index=False).agg(numRecords=('scenarioID','count'),minYear=('scenarioYear','min'),maxYear=('scenarioYear','max'),minMode=('MODE','min'))
dfRouteSummaryNames.to_csv(os.path.join((os.path.join(os.getcwd(),'intermediate')),'routesummarynames.csv'),index=False)
dfRouteSummaryNames

,NAME,numRecords,minYear,maxYear,minMode
0,1300So,10,2050,2050,4
1,1700So,22,2042,2050,5
2,200So_2,32,2024,2050,5
3,200So_Core,6,2030,2050,5
4,300W,32,2032,2050,5
...,...,...,...,...,...
209,StGrid_2,30,2030,2050,5
210,StGrid_2_BRT,24,2032,2050,9
211,StGrid_2_Core,4,2032,2032,5
212,TooeleCor,1,2050,2050,6


In [5]:
# read in manully created file
dfRouteSummaryNames_Revised = pd.read_csv(os.path.join((os.path.join(os.getcwd(),'manual')),'routesummarynames_revised.csv'))
dfRouteSummaryNames_Revised

,NAME_New,ROUTEID,LONGNAME,NAME,numRecords,minYear,maxYear,minMode,NAMEwithZeros
0,701_Blue,701,701 - TRAX Blue Line,Blue,62,2019,2050,7,Blue
1,703_Red,703,703 - TRAX Red Line,Red,64,2019,2050,7,Red
2,704_Green,704,704 - TRAX Green Line,Green,64,2019,2050,7,Green
3,720_SLine,720,720 - S-Line,Sline,62,2019,2050,7,Sline
4,750_FR,750,750 - FrontRunner,RCRT_WEPN,64,2019,2050,8,RCRT_WEPN
...,...,...,...,...,...,...,...,...,...
212,WFRC_082,NaN,WFRC_082 - StGrid_2,StGrid_2,30,2030,2050,5,StGrid_2
213,WFRC_083,NaN,WFRC_083 - StGrid_2_BRT,StGrid_2_BRT,24,2032,2050,9,StGrid_2_BRT
214,WFRC_084,NaN,WFRC_084 - StGrid_2_Core,StGrid_2_Core,4,2032,2032,5,StGrid_2_Core
215,WFRC_085,NaN,WFRC_085 - TooeleCor,TooeleCor,1,2050,2050,6,TooeleCor


In [14]:
dfRouteSummaryNames_Revised[['NAME_New','LONGNAME']].rename(columns={'NAME_New':'NAME'}).drop_duplicates().to_json(os.path.join((os.path.join(os.getcwd(),'manual')),'routesummarynames_revised.json'),orient='table')

In [7]:
# Modify VOYGER LIN Input Files